In [83]:
#pip install pandas sqlalchemy mysql-connector-python

ERROR: Could not find a version that satisfies the requirement nasdaq-datalink (from versions: none)
ERROR: No matching distribution found for nasdaq-datalink
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install pandas sqlalchemy mysql-connector-python
pip install prefect-multiprocess

  Using cached prefect_multiprocess-0.1.2-py3-none-any.whl.metadata (4.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pandas as pd
import nasdaqdatalink as nas
from sqlalchemy import create_engine
import os
import gzip
import pickle
import sqlalchemy as sa
import datatable as dt
import numpy as np
import quandl
from sqlalchemy import text
from io import BytesIO
from prefect import task, flow


here intend to use data in stock trding, and the data of stock price and fundamental data of the company are used

In [47]:
tickers = ["AAPL", "MSFT", "GOOGL"]
print(tickers[0])

AAPL


In [48]:
nas.ApiConfig.api_key = 'UCizQTvgM6noERXudhrx'
market_data = nas.get("WIKI/AAPL")
#Date column is properly formatted and has no duplicates, cannot be excuted twice or will generate another date column
market_data.index = pd.to_datetime(market_data.index)
market_data.reset_index(inplace=True)
market_data.rename(columns={'index': 'Date'}, inplace=True)
market_data['Daily_Return'] = market_data['Close'].pct_change()

print(market_data)

           Date    Open    High     Low    Close      Volume  Ex-Dividend  \
0    1980-12-12   28.75   28.87   28.75   28.750   2093900.0          0.0   
1    1980-12-15   27.38   27.38   27.25   27.250    785200.0          0.0   
2    1980-12-16   25.37   25.37   25.25   25.250    472000.0          0.0   
3    1980-12-17   25.87   26.00   25.87   25.870    385900.0          0.0   
4    1980-12-18   26.63   26.75   26.63   26.630    327900.0          0.0   
...         ...     ...     ...     ...      ...         ...          ...   
9395 2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
9396 2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
9397 2018-03-23  168.39  169.92  164.94  164.940  40248954.0          0.0   
9398 2018-03-26  168.07  173.10  166.44  172.770  36272617.0          0.0   
9399 2018-03-27  173.68  175.15  166.92  168.340  38962839.0          0.0   

      Split Ratio   Adj. Open   Adj. High    Adj. Low  Adj. Close  \
0     

In [49]:
#transform to datatable
dt_market_data = dt.Frame(market_data)
print(dt_market_data)

     | Date                    Open     High      Low    Close            Volume  Ex-Dividend  Split Ratio   Adj. Open   Adj. High    Adj. Low  Adj. Close  Adj. Volume  Daily_Return
     | time64               float64  float64  float64  float64           float64      float64      float64     float64     float64     float64     float64      float64       float64
---- + -------------------  -------  -------  -------  -------  ----------------  -----------  -----------  ----------  ----------  ----------  ----------  -----------  ------------
   0 | 1980-12-12T00:00:00    28.75    28.87    28.75   28.75        2.0939e+06             0            1    0.422706    0.42447     0.422706    0.422706  1.17258e+08    NA        
   1 | 1980-12-15T00:00:00    27.38    27.38    27.25   27.25   785200                      0            1    0.402563    0.402563    0.400652    0.400652  4.39712e+07    -0.0521739
   2 | 1980-12-16T00:00:00    25.37    25.37    25.25   25.25   472000                    

In [50]:
#fund_data = nas.get_table('ZACKS/FC', ticker = 'AAPL')
fund_data = nas.get_table('SHARADAR/SF1', ticker='AAPL')
#convert the date format for fundamental data
fund_data['datekey'] = pd.to_datetime(fund_data['datekey'])
fund_data.sort_values(by='datekey', inplace=True)
print(fund_data)
data_type = type(fund_data)
print(data_type)
#class 'pandas.core.frame.DataFrame' so just work on the dataframe type

     ticker dimension calendardate    datekey reportperiod lastupdated  \
None                                                                     
1      AAPL       MRY   2022-12-31 2022-09-24   2022-09-24  2024-06-07   
0      AAPL       MRY   2023-12-31 2023-09-30   2023-09-30  2024-06-07   

           accoci        assets     assetsavg       assetsc  ...    sharesbas  \
None                                                         ...                
1    -11109000000  352755000000  355229250000  135405000000  ...  16070752000   
0    -11452000000  352583000000  341632000000  143566000000  ...  15634232000   

         shareswa  shareswadil     sps     tangibles  taxassets       taxexp  \
None                                                                           
1     16215963000  16325819000  24.317  352755000000          0  19300000000   
0     15744231000  15812547000  24.344  352583000000          0  16741000000   

      taxliabilities   tbvps  workingcapital  
None      

In [51]:
#fetch US macro data
gdp_data = nas.get('FRED/GDP')
risk_free_rate_data = nas.get('FRED/DGS10')
unemployment_data = nas.get('FRED/UNRATE')

#convert the date format for each dataset
gdp_data.index = pd.to_datetime(gdp_data.index)
gdp_data.reset_index(inplace=True)
gdp_data.rename(columns={'index': 'Date', 'Value': 'GDP'}, inplace=True)

risk_free_rate_data.index = pd.to_datetime(risk_free_rate_data.index)
risk_free_rate_data.reset_index(inplace=True)
risk_free_rate_data.rename(columns={'index': 'Date', 'Value': 'Risk_Free_Rate'}, inplace=True)

unemployment_data.index = pd.to_datetime(unemployment_data.index)
unemployment_data.reset_index(inplace=True)
unemployment_data.rename(columns={'index': 'Date', 'Value': 'Unemployment_Rate'}, inplace=True)

#merge the datasets on the Date column by forward fulfilling
macro_data = pd.merge(gdp_data, risk_free_rate_data, on='Date', how='outer')
macro_data = pd.merge(macro_data, unemployment_data, on='Date', how='outer')
macro_data.fillna(method='ffill', inplace=True)

#print the merged dataset
print("Merged Macroeconomic Data:")
print(macro_data)
data_type = type(macro_data)
print(data_type)

Merged Macroeconomic Data:
            Date        GDP  Risk_Free_Rate  Unemployment_Rate
0     1947-01-01    243.164             NaN                NaN
1     1947-04-01    245.968             NaN                NaN
2     1947-07-01    249.585             NaN                NaN
3     1947-10-01    259.745             NaN                NaN
4     1948-01-01    265.742             NaN                3.4
...          ...        ...             ...                ...
15548 2022-07-11  23992.355            2.99                4.0
15549 2022-07-12  23992.355            2.96                4.0
15550 2022-07-13  23992.355            2.91                4.0
15551 2022-07-14  23992.355            2.96                4.0
15552 2022-07-15  23992.355            2.93                4.0

[15553 rows x 4 columns]
<class 'pandas.core.frame.DataFrame'>


/var/folders/kp/ss7596j53qzcgy076k8vjzhw0000gn/T/ipykernel_17600/2234416203.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  macro_data.fillna(method='ffill', inplace=True)


transform the 3 data into same format and in the same date index and merge them, using the method forward fulfill

In [52]:
print(macro_data)
print(market_data)

            Date        GDP  Risk_Free_Rate  Unemployment_Rate
0     1947-01-01    243.164             NaN                NaN
1     1947-04-01    245.968             NaN                NaN
2     1947-07-01    249.585             NaN                NaN
3     1947-10-01    259.745             NaN                NaN
4     1948-01-01    265.742             NaN                3.4
...          ...        ...             ...                ...
15548 2022-07-11  23992.355            2.99                4.0
15549 2022-07-12  23992.355            2.96                4.0
15550 2022-07-13  23992.355            2.91                4.0
15551 2022-07-14  23992.355            2.96                4.0
15552 2022-07-15  23992.355            2.93                4.0

[15553 rows x 4 columns]
           Date    Open    High     Low    Close      Volume  Ex-Dividend  \
0    1980-12-12   28.75   28.87   28.75   28.750   2093900.0          0.0   
1    1980-12-15   27.38   27.38   27.25   27.250    785200.0    

In [53]:
merged_data = pd.merge(market_data, macro_data, on='Date', how='left')

#forward fulfill
#merge fundamental data with the merged dataset, using forward fill method
merged_data = pd.merge_asof(merged_data, fund_data, left_on='Date', right_on='datekey', direction='backward')

#print the format of the final merged dataset
print("Final Merged Data Format:")
print(merged_data.info())

#print the final merged dataset
print("Final Merged Data:")
print(merged_data.tail())

Final Merged Data Format:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9400 entries, 0 to 9399
Columns: 128 entries, Date to workingcapital
dtypes: datetime64[ns](5), float64(121), object(2)
memory usage: 9.2+ MB
None
Final Merged Data:
           Date    Open    High     Low    Close      Volume  Ex-Dividend  \
9395 2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
9396 2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
9397 2018-03-23  168.39  169.92  164.94  164.940  40248954.0          0.0   
9398 2018-03-26  168.07  173.10  166.44  172.770  36272617.0          0.0   
9399 2018-03-27  173.68  175.15  166.92  168.340  38962839.0          0.0   

      Split Ratio  Adj. Open  Adj. High  ...  sharesbas  shareswa  \
9395          1.0     175.04     175.09  ...        NaN       NaN   
9396          1.0     170.00     172.68  ...        NaN       NaN   
9397          1.0     168.39     169.92  ...        NaN       NaN   
9398          1.0  

check data caveat: missing value: fill and delete; not matched: delete; repeating: delete; outliers: regression; format...

In [54]:
def check(data):
    #check for missing values
    def check_missing_values(data):
        missing_vals = data.isnull().sum()
        print("Missing Values:\n", missing_vals,'\n')
        return missing_vals

    #check for outliers (for daily return)
    def check_outliers(data, threshold=3):
        z_scores = (data['Daily_Return'] - data['Daily_Return'].mean()) / data['Daily_Return'].std()
        outliers = data[(np.abs(z_scores) > threshold)]
        print("Outliers:\n", outliers,'\n')
        return outliers

    #check for extreme price changes
    def check_extreme_price_change(data):
        extreme_change = data[abs((data['Close'] - data['Open']) / data['Open']) > 0.2]
        print("Extreme Price Changes:\n", extreme_change,'\n')
        return extreme_change

    #check for duplicates
    def check_duplicates(data):
        duplicates = data.duplicated().sum()
        print("Duplicates:\n", duplicates,'\n')
        return duplicates

    # Running all checks
    print("Initial Data Check:")
    check_missing_values(data)
    check_outliers(data)
    check_extreme_price_change(data)
    check_duplicates(data)


data = merged_data 
check(data)

Initial Data Check:
Missing Values:
 Date                 0
Open                 0
High                 0
Low                  0
Close                0
                  ... 
taxassets         9400
taxexp            9400
taxliabilities    9400
tbvps             9400
workingcapital    9400
Length: 128, dtype: int64 

Outliers:
            Date    Open    High     Low   Close      Volume  Ex-Dividend  \
197  1981-09-25   14.50   14.50   14.25   14.25    154500.0          0.0   
495  1982-11-30   28.87   32.00   28.75   31.88    710700.0          0.0   
531  1983-01-20   33.63   37.37   33.63   37.37   3160000.0          0.0   
686  1983-08-31   33.13   37.25   33.13   37.25    893900.0          0.0   
690  1983-09-07   39.25   39.25   33.87   34.62   1718100.0          0.0   
...         ...     ...     ...     ...     ...         ...          ...   
7012 2008-09-29  119.62  119.68  100.59  105.26  93644900.0          0.0   
7022 2008-10-13  104.55  110.53  101.02  110.26  54967000.0    

In [55]:
def clean(data):
    #handle missing values: Fill with forward fill method
    def handle_missing_values(data):
        data = data.fillna(method='ffill')
        return data
    """
    # Handle outliers: Replace outliers using mean value
    def handle_outliers(data, threshold=3):
        z_scores = (data['Daily_Return'] - data['Daily_Return'].mean()) / data['Daily_Return'].std()
        outliers = (np.abs(z_scores) > threshold)
        if outliers.any():
            mean_value = data.loc[~outliers, 'Daily_Return'].mean()
            data.loc[outliers, 'Daily_Return'] = mean_value
        return data
    """

    #drop duplicates
    def drop_duplicates(data):
        data = data.drop_duplicates()
        print("Duplicates dropped")
        return data

    #running all cleaning steps
    data = handle_missing_values(data)
    #data = handle_outliers(data)
    data = drop_duplicates(data)

    #print the cleaned dataset
    print("Cleaned Data:")
    print(data.info())
    print(data.tail())

    return data


cleaned_data = clean(data)


Duplicates dropped
Cleaned Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9400 entries, 0 to 9399
Columns: 128 entries, Date to workingcapital
dtypes: datetime64[ns](5), float64(123)
memory usage: 9.2 MB
None
           Date    Open    High     Low    Close      Volume  Ex-Dividend  \
9395 2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
9396 2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
9397 2018-03-23  168.39  169.92  164.94  164.940  40248954.0          0.0   
9398 2018-03-26  168.07  173.10  166.44  172.770  36272617.0          0.0   
9399 2018-03-27  173.68  175.15  166.92  168.340  38962839.0          0.0   

      Split Ratio  Adj. Open  Adj. High  ...  sharesbas  shareswa  \
9395          1.0     175.04     175.09  ...        NaN       NaN   
9396          1.0     170.00     172.68  ...        NaN       NaN   
9397          1.0     168.39     169.92  ...        NaN       NaN   
9398          1.0     168.07     173.10  ..

/var/folders/kp/ss7596j53qzcgy076k8vjzhw0000gn/T/ipykernel_17600/1591160712.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')
/var/folders/kp/ss7596j53qzcgy076k8vjzhw0000gn/T/ipykernel_17600/1591160712.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(method='ffill')


In [56]:
from sqlalchemy import create_engine

In [57]:
#in mySQL

CREATE DATABASE nasdaq_data;
USE nasdaq_data;
SELECT * FROM stock_data;
exit

#or could use a new account
CREATE USER 'myuser'@'localhost' IDENTIFIED BY 'mypassword';
GRANT ALL PRIVILEGES ON nasdaq_data.* TO 'myuser'@'localhost';
FLUSH PRIVILEGES;


SyntaxError: invalid syntax (493388394.py, line 3)

In [58]:
username = 'root'
password = 'NIdeshengri200!'
host = 'localhost'
database = 'nasdaq_data'

#link to database
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database}')

# search and load to dataframe
query = "SELECT * FROM stock_data" 
df = pd.read_sql(query, con=engine)


print(df.head())


    Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  Adj. Open  \
0  28.75  28.87  28.75  28.75  2093900.0          0.0          1.0   0.422706   
1  27.38  27.38  27.25  27.25   785200.0          0.0          1.0   0.402563   
2  25.37  25.37  25.25  25.25   472000.0          0.0          1.0   0.373010   
3  25.87  26.00  25.87  25.87   385900.0          0.0          1.0   0.380362   
4  26.63  26.75  26.63  26.63   327900.0          0.0          1.0   0.391536   

   Adj. High  Adj. Low  Adj. Close  Adj. Volume  
0   0.424470  0.422706    0.422706  117258400.0  
1   0.402563  0.400652    0.400652   43971200.0  
2   0.373010  0.371246    0.371246   26432000.0  
3   0.382273  0.380362    0.380362   21610400.0  
4   0.393300  0.391536    0.391536   18362400.0  


In [59]:
username = 'root'
password = 'NIdeshengri200!'
host = 'localhost'
database = 'nasdaq_data'
db_url = f'mysql+mysqlconnector://{username}:{password}@{host}/{database}'
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database}')

In [60]:
#data download; update and retrive

NASDAQ_API = 'UCizQTvgM6noERXudhrx'
nas.ApiConfig.api_key = NASDAQ_API

def download_data(source, ticker=None, **filters):
    """
    Download data from Nasdaq Data Link.
    
    Parameters:
    source (str): The data source or table name.
    ticker (str): The stock ticker symbol for the 'get' method (optional).
    **filters: Additional filters for the 'get_table' method.
    
    Returns:
    tuple: The source and the downloaded data as a DataFrame.
    """
    nas.ApiConfig.api_key = NASDAQ_API
    data = None
    
    try:
        # Try to fetch data using get_table if ticker is not provided
        if ticker is None:
            data = nas.get_table(source, **filters)
        else:
            # Try to fetch data using get if ticker is provided
            data = nas.get(f'{source}/{ticker}')
    except Exception as e:
        print(f"Error downloading data from {source} with ticker {ticker}: {str(e)}")
        return source, None

    if data is not None:
        print(f"Data downloaded successfully from {source} with ticker {ticker}")
    else:
        print(f"Failed to download data from {source} with ticker {ticker}")

    return source, data

table_name = '11111'
filters = {'ticker': 'AAPL', 'dimension': 'ARQ'}

result = download_data(table_name, **filters)
print(result)

source = 'WIKI'
ticker = 'AAPL'

result = download_data(source, ticker)
print(result)



def update_data(engine, table_name, source, ticker=None, **filters):

    table, new_data = download_data(source, ticker, **filters)
    
    if new_data is not None:
        #load existing data from the database
        existing_data = pd.read_sql_table(table_name, con=engine)
        
        #combine existing data with new data, dropping duplicates
        combined_data = pd.concat([existing_data, new_data]).drop_duplicates()
        
        #replace the table with the combined data
        combined_data.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        
        print(f"Data for {ticker} has been updated in the {table_name} table.")
    else:
        print(f"Failed to update data for {ticker} in the {table_name} table.")



def retrieve_data(engine, table_name):

    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, con=engine)
    return df



#download data and store in database
source = 'WIKI'
ticker = 'AAPL'
table, data = download_data(source, ticker)
if data is not None:
    data.to_sql(name='stock_data', con=engine, if_exists='replace', index=False)

#update data in the database
update_data(engine, 'stock_data', source, ticker)

#retrieve data from the database
retrieved_data = retrieve_data(engine, 'stock_data')
print(retrieved_data.head())


Error downloading data from 11111 with ticker AAPL: (Status 404) (Nasdaq Data Link Error QECx02) You have submitted an incorrect Dataset code. Please check your Dataset codes and try again.
('11111', None)
Data downloaded successfully from WIKI with ticker AAPL
('WIKI',               Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
1980-12-12   28.75   28.87   28.75   28.750   2093900.0          0.0   
1980-12-15   27.38   27.38   27.25   27.250    785200.0          0.0   
1980-12-16   25.37   25.37   25.25   25.250    472000.0          0.0   
1980-12-17   25.87   26.00   25.87   25.870    385900.0          0.0   
1980-12-18   26.63   26.75   26.63   26.630    327900.0          0.0   
...            ...     ...     ...      ...         ...          ...   
2018-03-21  175.04  175.09  171.26  171.270  35247358.0          0.0   
2018-03-22  170.00  172.68  168.60  168.845  41051076.0          0.0   
2018-03-2

In [61]:
#Store DataFrame to the database.

def store_df_to_db(df, engine, table_name):
    """
    Parameters:
    df (DataFrame): The DataFrame to store.
    table_name (str): The name of the table where the DataFrame will be stored.
    """
    
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"DataFrame has been stored in the {table_name} table.")

store_df_to_db(data, engine, 'stock_data')
retrieved_data = retrieve_data(engine, 'stock_data')#read
print(retrieved_data.head())


DataFrame has been stored in the stock_data table.
    Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  Adj. Open  \
0  28.75  28.87  28.75  28.75  2093900.0          0.0          1.0   0.422706   
1  27.38  27.38  27.25  27.25   785200.0          0.0          1.0   0.402563   
2  25.37  25.37  25.25  25.25   472000.0          0.0          1.0   0.373010   
3  25.87  26.00  25.87  25.87   385900.0          0.0          1.0   0.380362   
4  26.63  26.75  26.63  26.63   327900.0          0.0          1.0   0.391536   

   Adj. High  Adj. Low  Adj. Close  Adj. Volume  
0   0.424470  0.422706    0.422706  117258400.0  
1   0.402563  0.400652    0.400652   43971200.0  
2   0.373010  0.371246    0.371246   26432000.0  
3   0.382273  0.380362    0.380362   21610400.0  
4   0.393300  0.391536    0.391536   18362400.0  


In [62]:
import os
import zlib
import pickle

In [63]:
#Compress and save the DataFrame to a specified file.


def compress_and_save_df(df, save_path='.', file_name='compressed_data.pkl.gz'):
    """
    Parameters:
    df (DataFrame): The DataFrame to compress and save.
    save_path (str): The directory path to save the file. Default is current directory.
    file_name (str): The name of the file. Default is 'compressed_data.pkl.gz'.
    
    Returns:
    bytes: The compressed data.
    """
    os.makedirs(save_path, exist_ok=True)
    file_path = os.path.join(save_path, file_name)
    
    with gzip.open(file_path, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"DataFrame has been compressed and saved to {file_path}.")
    return file_path


compressed_file_path = compress_and_save_df(data, save_path='encoded_folder', file_name='data.pkl.gz')


DataFrame has been compressed and saved to encoded_folder/data.pkl.gz.


In [ ]:
#sql if too long
ALTER TABLE compressed_data MODIFY COLUMN data MEDIUMBLOB; 


In [64]:
def store_compressed_data_to_db(file_path, engine, table_name):

    with engine.connect() as conn:
        conn.execute(text(f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                id INT AUTO_INCREMENT PRIMARY KEY,
                data MEDIUMBLOB
            );
        """))#Make sure database columns are large enough to store compressed data

    metadata = sa.MetaData()
    compressed_table = sa.Table(table_name, metadata, autoload_with=engine)

    with open(file_path, 'rb') as f:
        compressed_data = f.read()
    print(f"Compressed data read from file: {len(compressed_data)} bytes")
    print(compressed_data)
    with engine.connect() as conn:
        insert_stmt = compressed_table.insert().values(data=compressed_data)
        result = conn.execute(insert_stmt)
        conn.commit()#used to commit the current transaction, ensuring that all changes to the database are persisted.
        record_id = result.inserted_primary_key[0]
    
    print(f"Compressed data has been stored in the {table_name} table with record ID {record_id}.")
    print(result)
    return record_id

file_path = 'encoded_folder/data.pkl.gz'
store_compressed_data_to_db(file_path, engine, 'compressed_data')


Compressed data read from file: 414590 bytes
b'\x1f\x8b\x08\x08lx}f\x02\xffdata.pkl\x00\x8c\x9dy`\xddU\x95\xf8\xbf\x8a\xa2 J\x15\x15\x01\x8d\xaf*k\xb7\xb4I\xb34I\xf3\xcd\xd6to\xba\xef\xe9K\xf7Bi\x03\xdd\x17\xdao[\xaa U\x82\x14A\xc4\xf8\x10\x15\x1c\x19\xa8\x83\x8c\xb8?q\x10\xfd\xc9`Q\x94E\'>\x06Q\x1c\x1d\x8d\xa3(2\x08\xbf\x90\xf39\xf7\xe6\x9e\xf7\xbet\xf2GO\xef}w9\xf7\xdcs\xcf=\xcb\xbd\xf7\x9b\xbc\xee\x86\xb7\xbd&\x1a\xfc;\xf2\xd6\xee\xaeM\xab\xbb\xb6\x8c^\xb5\xf9\xf25\xa3\xd7^\xdeu\xe9\x9a\xa3GNi\xed\xda\xda5i\xf0\xff\xd7\x1f\xbd\xf0\xc0\xd1+\x8e^p\xe4u+.]w\xf9\xd1#eC\x8bo\xd8\xb4u\xcd\xe5\x9b\xba6n\x19}i\xd7\xa6\xaeuk.\xdfr\xf4\xc8i\xcd\x1b7\xaf\xbad\x86\xa4\x07\xaa\x1fy\'5Vl\xdc\xb0r\x8b\xafr\xf4\xc8\xe9+\xb6m\xea\xde\xb0\xea\x92\x8dkV\xac|\xa5\xce+\x85\xdf\xb1i\xdb\xa5\xdd\xbb\xa4\xf5K\xb7m\xdc\xba\xa1\xeb\xf2\xcb\xbbv\r\xb4\xba\xe2\xf25\xab6o\xda\xb2\xf5\xf2m\xab\xb6\xbeR\xf0\xf5\x83\x05\x8f\x1ey\xc3@\xdb\x83E\xae?:-:|\xb4\xe55+\x8f~\xe8\xe8\x9c\xa3\x17L{\xcd\xb4\xd3f\xdc\xf7\x81\x

22

In [ ]:
print(store_compressed_data_to_db(file_path, engine, 'compressed_data'))

Compressed data has been stored in the compressed_data table with record ID 5.
5


In [65]:
def load_compressed_data_from_db(engine, table_name, record_id):
    """
    Load and decompress data from the database.
    
    Parameters:
    table_name (str): The name of the table where the compressed data is stored.
    record_id (int): The ID of the record to load.
    
    Returns:
    DataFrame: The decompressed DataFrame.
    """
        
    metadata = sa.MetaData()
    compressed_table = sa.Table(table_name, metadata, autoload_with=engine)
    
    with engine.connect() as conn:
        #Perform a raw SQL query
        query = f"SELECT data FROM {table_name} WHERE id = :record_id"
        result = conn.execute(sa.text(query), {"record_id": record_id}).fetchone()
        if result is None:
            raise ValueError(f"No record found with ID {record_id}")
        
        compressed_data = result[0]
        print(f"Compressed data length: {len(compressed_data)} bytes")  # Add debug print
    
    with gzip.open(BytesIO(compressed_data), 'rb') as f:
        df = pickle.load(f)
    
    return df




record_id = store_compressed_data_to_db(file_path, engine, 'compressed_data')
print(f"Stored record ID: {record_id}")

# Then, load the compressed data
try:
    decompressed_df = load_compressed_data_from_db(engine, 'compressed_data', record_id)
    print(decompressed_df.head())
except ValueError as e:
    print(e)


Compressed data read from file: 414590 bytes
b'\x1f\x8b\x08\x08lx}f\x02\xffdata.pkl\x00\x8c\x9dy`\xddU\x95\xf8\xbf\x8a\xa2 J\x15\x15\x01\x8d\xaf*k\xb7\xb4I\xb34I\xf3\xcd\xd6to\xba\xef\xe9K\xf7Bi\x03\xdd\x17\xdao[\xaa U\x82\x14A\xc4\xf8\x10\x15\x1c\x19\xa8\x83\x8c\xb8?q\x10\xfd\xc9`Q\x94E\'>\x06Q\x1c\x1d\x8d\xa3(2\x08\xbf\x90\xf39\xf7\xe6\x9e\xf7\xbet\xf2GO\xef}w9\xf7\xdcs\xcf=\xcb\xbd\xf7\x9b\xbc\xee\x86\xb7\xbd&\x1a\xfc;\xf2\xd6\xee\xaeM\xab\xbb\xb6\x8c^\xb5\xf9\xf25\xa3\xd7^\xdeu\xe9\x9a\xa3GNi\xed\xda\xda5i\xf0\xff\xd7\x1f\xbd\xf0\xc0\xd1+\x8e^p\xe4u+.]w\xf9\xd1#eC\x8bo\xd8\xb4u\xcd\xe5\x9b\xba6n\x19}i\xd7\xa6\xaeuk.\xdfr\xf4\xc8i\xcd\x1b7\xaf\xbad\x86\xa4\x07\xaa\x1fy\'5Vl\xdc\xb0r\x8b\xafr\xf4\xc8\xe9+\xb6m\xea\xde\xb0\xea\x92\x8dkV\xac|\xa5\xce+\x85\xdf\xb1i\xdb\xa5\xdd\xbb\xa4\xf5K\xb7m\xdc\xba\xa1\xeb\xf2\xcb\xbbv\r\xb4\xba\xe2\xf25\xab6o\xda\xb2\xf5\xf2m\xab\xb6\xbeR\xf0\xf5\x83\x05\x8f\x1ey\xc3@\xdb\x83E\xae?:-:|\xb4\xe55+\x8f~\xe8\xe8\x9c\xa3\x17L{\xcd\xb4\xd3f\xdc\xf7\x81\x

In [ ]:
from prefect import task, flow


@task
def extract():
    data = {"name": "Prefect", "type": "workflow automation"}
    return data

@task
def transform(data):
    data["type"] = data["type"].upper()
    return data

@task
def load(data):
    print(f"Loading data: {data}")


@flow
def etl_flow():
    data = extract()
    transformed_data = transform(data)
    load(transformed_data)


etl_flow()


/Users/bryanwang/miniconda3/envs/myenv_x86/lib/python3.11/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_flow_run__coalesce_start_time_expected_start_time_desc
  next(self.gen)
/Users/bryanwang/miniconda3/envs/myenv_x86/lib/python3.11/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_flow_run__coalesce_start_time_expected_start_time_asc
  next(self.gen)


21:24:44.199 | INFO    | prefect.engine - Created flow run 'resourceful-caiman' for flow 'etl-flow'

21:24:44.243 | INFO    | Flow run 'resourceful-caiman' - Created task run 'extract-0' for task 'extract'

21:24:44.244 | INFO    | Flow run 'resourceful-caiman' - Executing 'extract-0' immediately...

21:24:44.284 | INFO    | Task run 'extract-0' - Finished in state Completed()

...

21:24:44.349 | INFO    | Flow run 'resourceful-caiman' - Executing 'load-0' immediately...

Loading data: {'name': 'Prefect', 'type': 'WORKFLOW AUTOMATION'}


21:24:44.385 | INFO    | Task run 'load-0' - Finished in state Completed()

21:24:44.399 | INFO    | Flow run 'resourceful-caiman' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `dict`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `dict`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]